In [2]:
import numpy as np
from importlib import reload
from scipy import signal

import plotting_tools as pt
import utils
utils = reload(utils)
pt = reload(pt)

import dill as pickle
import sys
import os
# Notebook needs to see the RKLM_Python module to load pickled class instances
sys.path.append('../modules/')
sys.path.append('../RKLM_Python/')

import matplotlib.pyplot as plt
from matplotlib import rc

plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath,amsfonts,amssymb,bm,mathrsfs}')
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.style.use('ggplot')

import matplotlib.colors as mcolors
import matplotlib as mtl

## Helper functions to load and transform solutions

In [11]:
i2 = (slice(2,-2),slice(2,-2))
# i2 = (slice(0,-2),slice(0,-2))

# i2x = (slice(2,-2),slice(0,-1))
# i2x = (slice(0,-2),slice(0,-1))

# i1 = (slice(2,-2))
# i1 = (slice(0,-2))

inner = False

def load(Nx,Ny,et,test):
    if test == 'mark':
        # base_fn = 'output'
        base_fn = 'output_mark_wave'
    # pydir = '../%s/' %base_fn
    pydir = '/home/ray/git-projects/RKLM_Reference/output_mark_wave/'

    tc = utils.test_case(base_fn, pydir, Nx, Ny, et)
    tags = tc.get_tag_dict()
    
    return tc, tags

def get_ens(tc, times, sfx , diff, attribute, tag, SIDX, NIDX):
    idx = 0 if tag == 'ic' else -1
    times = [0] if tag == 'ic' else times
    ens = tc.get_ensemble(times, N, attribute, sfx, label_type=l_typ, avg=True, diff=diff, tag=tag, inner=inner,load_ic=True)[idx]

    label = sfx + '_' + attribute
    return label, ens.T[SIDX:NIDX,:]

def get_ens_all(tc, times, sfx , diff, attribute, tag, SIDX, NIDX):
    ens = tc.get_ensemble(times, N, attribute, sfx, label_type=l_typ, avg=True, diff=diff, tag=tag, inner=inner,load_ic=True)
    ens = ens[:,:,SIDX:NIDX]
    return ens

# here, we load the time series of the simulations
def get_tc(typ, times, Nx, Ny, NIDX, tp='half', test='mark', et=8000.0):
    sfx1 = '%s' %typ

    tc, tags = load(Nx,Ny,et,test)
    if tp=='afs':
        tag = 'after_full_step'
    else:
        tag = tp

    # load pickled instances of data used in simulation
    fn_pickle = tc.get_filename(N,sfx1,format='dat')
    path_pickle = tc.get_path(fn_pickle)

#     print(path_pickle)
    file = open(path_pickle,'rb')
    ud = pickle.load(file)
    mpv = pickle.load(file)
    elem = pickle.load(file)
    node = pickle.load(file)
    dap = pickle.load(file)
    file.close()

    _, rho = get_ens(tc, times, sfx1, diff, 'rho', tag, 0, NIDX)
    
    return (ud,mpv,elem), tc

# function to recover the transformed variables in dimensional form
def hatter(arrs, ud, mpv, elem, SIDX, NIDX):
    arrs_rhou, arrs_rhov, arrs_rhoY, arrs_pi, rhos = arrs
    # rhobar = mpv.HydroState.rho0[2:-2][SIDX:NIDX]
    rhobar = mpv.HydroState.rho0[SIDX:NIDX]
    uhat = ((arrs_rhou / rhos) - ud.u_wind_speed) * rhobar**0.5 * (ud.u_ref * np.sqrt(ud.rho_ref))
    vhat = arrs_rhov / rhos * rhobar**0.5 * (ud.u_ref * np.sqrt(ud.rho_ref))
    
    vhat = arrs_rhov / rhos
    Nfq = ud.Nsq_ref**0.5
    g = 9.81
    
    # Ybar = mpv.HydroState.Y0[2:-2][SIDX:NIDX]
    # S0 = (ud.stratification(elem.y)).reshape(1,1,-1)[:,:,2:-2][:,:,SIDX:NIDX]
    
    Ybar = mpv.HydroState.Y0[SIDX:NIDX]
    S0 = (ud.stratification(elem.y)).reshape(1,1,-1)[:,:,SIDX:NIDX]

    Yhat = g / Nfq * (rhobar**0.5 * np.sqrt(ud.rho_ref)) / Ybar * (arrs_rhoY / rhos - Ybar)
    
    Yhat = arrs_rhoY / rhos
    uhat = (arrs_rhou / rhos) / Yhat
    
    p2c = np.array([signal.fftconvolve(mem,[[1,1],[1,1]], mode='valid')/4.0 for mem in arrs_pi])
    C = np.sqrt(1.4 * ud.u_ref**2/ud.Msq)
    p2hat = ud.cp_gas / C * (rhobar**0.5 * np.sqrt(ud.rho_ref)) * Ybar * p2c * ud.T_ref * ud.Msq
    p2hat = p2c
    
    return uhat, vhat, Yhat, p2hat

## Output solution plot for each prognostic variable over the first 30 time steps.

In [96]:
l_typ = 'WINDOW_STEP'
diff = False
attr_labels = pt.labels()

tp ='afs'
tag = 'after_full_step'
time_unit = 's'
N = 1

# times0 = np.arange(30)
# times0 = np.arange(980,1000)
# times0 = np.arange(665,673)
times0 = np.arange(-1,7200,100)[1:]
# times0 = np.arange(1,11,1)-1
# times0 = [0,999,1999,2999,3999,4999,5999,6999]
times0 = np.append([0,0], times0)
# times0 = np.arange(31,50)

# choose spatial resolution (Nxs, Nys) and temporal resolution (Nts)
# Nxs = [151]
# Nys = [60]
Nxs = [301]
Nys = [10]
# Nxs = [601]
# Nys = [240]
# Nxs = [1201]
# Nys = [480]
# Nts = [200,400,600,800,1000,1200,1400,1600]
Nts = [10]
# Nts = [10.0]
# Nts = [1200]
# Nts = [1000,1200,1400,1600]
# oms = ['', '_noom']
# oms = ['']
oms = ['_noom']

# level of the contours
lu = [-0.06, -0.6, -6.0][::-1] + [-0.01, 0.01] + [0.06, 0.6, 6.0]
lv = [-0.04, -0.4, -4.0][::-1] + [-0.01, 0.01] + [0.04, 0.4, 4.0]
lrhoY = [-0.02, -0.2, -2.0][::-1] + [-0.01, 0.01] + [0.02, 0.2, 2.0]


# lu = [-0.000024, -0.00024, -2.4][::-1] + [-1e-6, 1e-6] + [0.000024, 0.00024, 2.4]
# lv = [-0.001, -0.01, -1.0][::-1] + [-1e-6, 1e-6] + [0.001, 0.01, 1.0]
# lrhoY = [-0.002, -0.02, -2.0][::-1] + [-1e-6, 1e-6] + [0.002, 0.02, 2.0]



# lrhoY = np.array(lrhoY)
# lrhoY = 10**lrhoY / 0.01
# lrhoY = np.log(lrhoY) + np.log(0.01)
# print(lrhoY)
lp2n = lu
lvls = np.array([lu,lv,lrhoY,lp2n])
# lvls = lvls*1e-5

# ref: https://stackoverflow.com/questions/61897393/unevenly-irregularly-spaced-data-for-colorbar-with-evenly-spaced-colors
cmap_rb = plt.get_cmap('cubehelix')
lu_colors = cmap_rb(np.linspace(0, 1, len(lu) + 1))
lu_cm, lu_nm = mcolors.from_levels_and_colors(lu, lu_colors, extend='both')

lv_colors = cmap_rb(np.linspace(0, 1, len(lv) + 1))
lv_cm, lv_nm = mcolors.from_levels_and_colors(lv, lv_colors, extend='both')

lrhoY_colors = cmap_rb(np.linspace(0, 1, len(lrhoY) + 1))
lrhoY_cm, lrhoY_nm = mcolors.from_levels_and_colors(lrhoY, lrhoY_colors, extend='both')

lp2n_colors = cmap_rb(np.linspace(0, 1, len(lp2n) + 1))
lp2n_cm, lp2n_nm = mcolors.from_levels_and_colors(lp2n, lp2n_colors, extend='both')
cmaps = [[lu_cm,lu_nm],[lv_cm,lv_nm],[lrhoY_cm,lrhoY_nm],[lp2n_cm,lp2n_nm]]

for Nx, Ny in zip(Nxs,Nys):  
    for Nt in Nts:
        for om in oms:
            typ = 'bdl_run_S%i%s_a05' %(Nt,om)
            # typ = 'bdl_test_S%i%s_a1' %(Nt,om)
            ics,tc = get_tc(typ,times0[1:], Nx, Ny, Ny, tp=tp, et=720.0)
            
            ud, mpv, elem = ics[0], ics[1], ics[2]
            SIDX = np.abs(elem.y - 0.0).argmin()
            NIDX = np.abs(elem.y - 2.5).argmin()
            print(elem.y[NIDX])
            # NIDX = 121

            t_axs = np.append(0,times0)
            phys_time = tc.t_arr
            # phys_time = np.append(phys_time)
            # print(phys_time)

            if '_noom_' in typ:
                tcase = 'wo_omega' ## \Omega = 0.0, traditional setting
                tstr = r'0.0'
                fld = 'noom'
            else:
                tcase = 'a05' ## Non-tradtional setting
                tstr = '%.6f' %(ud.coriolis_strength[2] / ud.t_ref/ 2.0)
                fld = 'om'

                
            cnt = 0
            ic_not_loaded = True
            for idx,time in enumerate(times0):
                # print("times0 = ", times0)
                if ((int(time) == 0) and (ic_not_loaded)):
                    tag = 'ic' 
                    ic_not_loaded = False
                    t_name = 0
                else:
                    tag = 'after_full_step'
                    t_name = time+1
                    
                tt = [time]
                rhos = get_ens_all(tc, tt, typ, False, 'rho', tag, SIDX, NIDX)[:]
                arrs_rhou = get_ens_all(tc, tt, typ, False, 'rhou', tag, SIDX, NIDX)[:]
                arrs_rhov = get_ens_all(tc, tt, typ, False, 'rhov', tag, SIDX, NIDX)[:]
                arrs_rhoY = get_ens_all(tc, tt, typ, False, 'rhoY', tag, SIDX, NIDX)[:]
                arrs_pi = get_ens_all(tc, tt, typ, False, 'p2_nodes', tag, SIDX, NIDX+1)[:]

                arrs = [arrs_rhou, arrs_rhov, arrs_rhoY, arrs_pi, rhos]
                arrs_hat = hatter(arrs, ud, mpv, elem, SIDX, NIDX)
                slc_idx = 0 if tt == 0 else -1
                arrs_hat = [arr[slc_idx,...].T for arr in arrs_hat]
                uhat, vhat, Yhat, p2hat = arrs_hat

                ud, mpv = ics[0], ics[1]
                ct = 0.0 if time == 0 else time
                
                if tag == 'ic':
                    print(Yhat.max(),Yhat.min())

                # pl_lst = [[uhat, r"$\chi_{u}$"],[vhat, r"$\chi_{w}$"],[Yhat, r"$\chi_{\theta}$"],[p2hat, r"$\chi_{\pi}$"]]
                pl_lst = [[uhat, r"${u/\theta}$"],[vhat, r"${w}$"],[Yhat, r"${\theta}$"],[p2hat, r"${\pi}$"]]
                pl = pt.plotter(pl_lst,ncols=2,figsize=(30,16),sharey=False, fontsize=32)
                x_axs = np.array(np.linspace(ud.xmin,ud.xmax,7))
                y_axs = np.array(np.linspace(elem.y[SIDX],elem.y[NIDX-1],6))

                x_axs = np.around(x_axs,2) * 10
                y_axs = np.around(y_axs,1)

                x_loc = np.linspace(0,(Nx-1),7)
                y_loc = np.linspace(0,uhat.shape[0]-1,6)
                y_axs = np.array([0.0,0.5,1.0,1.5,2.0,2.5]) * 10.0
                
                if NIDX == 121:
                    y_loc = np.linspace(0,NIDX-SIDX-1,5)
                    y_axs = np.array([0.0,2.0,4.0,6.0,8.0]) * 10.0

                x_label = r'$x$ [km]'
                y_label = r'$z$ [km]'

                pl.set_axes(x_locs=x_loc, y_locs=y_loc, x_axs=x_axs, y_axs=y_axs, x_label=x_label, y_label=y_label)

                tu = ud.t_ref#/3600
                print([t_name], phys_time[cnt]*tu)
                stitle = r'$t=$%.4f~s; (%i,%i); $\Delta t=$%.2f~s; $\Omega_y=$ %s~s$^{-1}$' %(phys_time[cnt]*tu, Nx,Ny,Nt,tstr)
                _ = pl.plot(method='contour', lvls=None, suptitle=stitle, rect=[0, 0.03, 1, 0.95])
                # _ = pl.plot(method='contour', suptitle=stitle, rect=[0, 0.03, 1, 0.95])
                fld_pth = './output_plot/%ix%i/S%i/%s' %(Nx,Ny,Nt,fld)
                if not os.path.exists(fld_pth):
                    os.makedirs(fld_pth)
                    
                pl.save_fig('%s/lamb_%s_%.3d' %(fld_pth, tcase, t_name), format='.png')
                # plt.show()

                plt.close()
                cnt += 1
    
print(typ)

2.375
1.9947606 0.95991516
[0] 0.0
[1] 10.0
[100] 999.9999999999981
[200] 2000.0000000000014
[300] 3000.0000000000155
[400] 4000.00000000003
[500] 5000.000000000044
[600] 6000.000000000058
[700] 7000.00000000003
[800] 7999.999999999973
[900] 8999.999999999916
[1000] 9999.99999999986
[1100] 10999.999999999802
[1200] 11999.999999999745
[1300] 12999.99999999969
[1400] 13999.999999999633
[1500] 14999.999999999576
[1600] 15999.99999999952
[1700] 16999.99999999946
[1800] 17999.999999999407
[1900] 18999.99999999935
[2000] 19999.99999999929
[2100] 20999.999999999236
[2200] 21999.999999999178
[2300] 22999.999999999123
[2400] 23999.999999999065
[2500] 24999.999999999007
[2600] 25999.99999999906
[2700] 26999.99999999929
[2800] 27999.999999999516
[2900] 28999.999999999745
[3000] 29999.99999999997
[3100] 31000.0000000002
[3200] 32000.000000000426
[3300] 33000.000000000655
[3400] 34000.00000000088
[3500] 35000.000000001106
[3600] 36000.00000000134
[3700] 37000.000000001564
[3800] 38000.00000000179
[

In [65]:
print(Nts)

[600]


## Compute relative norm plot for the instability growth rate against time

### Helper functions

In [9]:
def get_norm(typ, times, Nx, Ny, NIDX, SIDX=None):
    ics,tc = get_tc(typ,times, Nx, Ny, NIDX, tp=tp, et=720.0)
    ud, mpv, elem = ics
    tt = times
    
    didx = np.abs(elem.y[2:-2] - 0.0).argmin()
    if SIDX is None: 
        SIDX = didx
        NIDX = np.abs(elem.y[2:-2] - 2.5).argmin()
    else:
        NIDX = np.abs(elem.y[2:-2] - 2.5).argmin()
    print("SIDX; NIDX; elem.y at SIDX; elem.y at NIDX: ", SIDX, NIDX, elem.y[2:-2][SIDX], elem.y[2:-2][NIDX])
    
    rhos = get_ens_all(tc, tt, typ, False, 'rho', tag, SIDX, NIDX)[:]
    arrs_rhou = get_ens_all(tc, tt, typ, False, 'rhou', tag, SIDX, NIDX)[:]
    arrs_rhov = get_ens_all(tc, tt, typ, False, 'rhov', tag, SIDX, NIDX)[:]
    arrs_rhoY = get_ens_all(tc, tt, typ, False, 'rhoY', tag, SIDX, NIDX)[:]
    arrs_pi = get_ens_all(tc, tt, typ, False, 'p2_nodes', tag, SIDX, NIDX+1)[:]
    
    ################################
    # w/ Dimensions
    ################################
    
    arrs = [arrs_rhou, arrs_rhov, arrs_rhoY, arrs_pi, rhos]
    uhat, vhat, Yhat, p2hat = hatter(arrs, ud, mpv, elem, SIDX, NIDX)

    t_axs = np.array(tc.t_arr)
    tt = t_axs * ud.t_ref

    res = []
    
#     print(arrs_rhou.shape)
    for idx, arr in enumerate(arrs_rhou):
            
        u = uhat[idx]
        v = vhat[idx]
        Y = Yhat[idx]
        p = p2hat[idx]
        nnorm = np.array([u,v,Y,p])
        if idx == 0:
            E0 = np.copy(la.norm(nnorm))
        res.append(la.norm(nnorm) / E0)

    res = np.array(res)
    return res, ud, tt


def looper(Nxs,Nys,Nts,params,fs=(5,4),ftsz=10):
    pidx = 0
    nexps = []
    for Nx, Ny in zip(Nxs,Nys):
        for Nt in Nts:
            typ1 = 'bdl_run_S%i_a05' %Nt
            # typ1 = 'bdl_test_S%i_a1' %Nt
            typ2 = 'bdl_run_S%i_noom_a05' %Nt
            NIDX = int((Ny/4))

            # nts = 45 if (Nx == 151 and Ny == 60 and Nt == 1600) else 50
            nts = 101
            times0 = np.arange(nts)
            
            SIDX = np.abs(elem.y[2:-2] - 0.0).argmin()

            res_om, ud_om, tt_om = get_norm(typ1, times0, Nx, Ny, NIDX, SIDX=SIDX)
            res_full_om, ud_full_om, tt_full_om = get_norm(typ1, times0, Nx, Ny, NIDX, SIDX=None)
            res_noom, ud_noom, tt_noom = get_norm(typ2, times0, Nx, Ny, NIDX)

            fig = plt.figure(figsize=(fs))
            fig.add_subplot(111)
            ax = plt.gca()

            tu_om = ud_om.t_ref
            tu_noom = ud_noom.t_ref
            
            if Nt < 600:
                me_om, me_noom = 2, 3
            else:
                me_om, me_noom = 1, 1
            ax.semilogy(tt_om, res_om, '-o', lw=2, markevery=me_om, label=r'$\Omega_y = %.3f \times 10^{-5}~$s$^{-1}$; $z \in [0.0,25.0]~$km' %(ud_om.coriolis_strength[2] * 1e5 / tu_om / 2.0))
            # ax.semilogy(tt_full_om, res_full_om, '-', c='C3', alpha=0.75, lw=2, markevery=me_om, label=r'$\Omega_y = %.3f \times 10^{-5}~$s$^{-1}$; $z \in [0.0,25.0]~$km' %(ud_om.coriolis_strength[2] * 1e5 / tu_om / 2.0))
            ax.semilogy(tt_noom, res_noom, '--o', lw=2, markevery=me_noom, label=r'$\Omega_y = 0.0~$s$^{-1}$')
            
            
            tts, tte = params[pidx]
            res_rg = res_om[tts:tte+1]
            tt_rg = tt_om[tts:tte+1]
        
            grad, zb = np.polyfit(tt_rg, np.log(res_rg), 1)

            ttfit = np.linspace(0,(tt_om[tte]-tt_om[tts]),5)
            
            prep_grad = np.around(grad * 1e4,2)
            print("grid =", prep_grad)
            lgrad = np.log10(grad)
            print("lgrad =", lgrad)
            zbb = grad * tt_om[tts] + zb
            zbb = np.exp(zbb)
            fitted = zbb*np.exp(10.0**(lgrad)*ttfit)

            ttf = ttfit
            ttf = ttf + tt_om[tts]

            ax.semilogy(ttf, fitted, 'k--', lw=2, label=r'$\mathscr{F}(t) = \mathscr{F}_0 \exp(%.2f \times 10^{-4}~t)$' %prep_grad)

            formatter = ticker.ScalarFormatter()
            formatter.set_scientific(True)
            formatter.set_powerlimits((-1,1))
            ax.yaxis.set_major_formatter(formatter)

            ymin, ymax = plt.gca().get_ylim()

            ax.set_title("Spatial resolution (Nx, Nz): (%i, %i)" %(Nx,Ny))
            ax.set_ylabel(r'$\log_{10}$[rel. norm]', fontsize=ftsz)
            t_units = 's'
            ax.set_xlabel("time $t$ [%s]" %t_units, fontsize=ftsz)

            ax.legend(fontsize=int(ftsz-1))
            ax.set_xlim([0.0,tt_om.max()])
            xtl = ax.get_xticks()
            ytl = ax.get_yticks()
            ax.set_xticklabels(xtl, fontsize=ftsz)
            ax.set_yticklabels(ytl, fontsize=ftsz)
            fld_pth = './output_plot/%ix%i/S%i' %(Nx,Ny,Nt)
            if not os.path.exists(fld_pth):
                os.makedirs(fld_pth)
            plt.savefig('%s/plot.png' %fld_pth)
            plt.tight_layout()
            plt.show()

            pidx += 1
            
            nexps.append(lgrad)
            
        print(typ1)
    return nexps


### Semilogy plot of the relative norm against time

In [10]:
## from matplotlib.ticker import (FixedLocator)
from matplotlib import ticker
import matplotlib

plt.style.use('ggplot')

from numpy import linalg as la
import csv

l_typ = 'WINDOW_STEP'
diff = False

attr_labels = pt.labels()

# Nxs = [151,301,601,1201]
# Nys = [60,120,240,480]
# Nxs = [120,120]
# Nys = [120,240]
# Nxs = [151]
# Nys = [60]
Nxs = [301]
Nys = [40]
# Nxs = [601]
# Nys = [240]
# Nxs = [1201]
# Nys = [480]
# Nxs = [151, 301, 601, 1201]
# Nys = [60, 120, 240, 480]
# Nts = [200,400,600,800,1000,1200,1400,1600]
Nts = [600]
# Nts = [200,400,600,800,1000]
# Nts = [400,600,800]
# Nts = [400]
# Nts = [1200,1400,1600]

N = 1
et = 720.0
nts = 100

tp = 'afs'
attribute = 'rhou'
tag = 'after_full_step'

# choose which data points to fit.
params = [
# 151x60:
        [34,36],
        [14,16],
        [8,11],
        [4,8],
        [4,7],
        [4,5],
        [4,5],
        [4,5],
# 301x120:
        [15,34],
        [6,9],
        [3,5],
        [3,4],
        [3,4],
        [3,4],
        [3,4],
        [2,4],
# 601x240:
        [9,17],
        [3,5],
        [2,4],
        [2,3],
        [2,3],
        [2,3],
        [2,3],
        [2,3],
# 1201x480:
        [5,7],
        [2,4],
        [2,3],
        [2,3],
        [2,3],
        [2,3],
        [2,3],
        [2,3],
         ]

params = [(8,9)]
params = [(10,35)]
# params = [(25,25)]

nexps = looper(Nxs,Nys,Nts,params,fs=(8,5),ftsz=14)
print(nexps)

output = False
if output:
    labels = []
    for Nx, Ny in zip(Nxs,Nys):
        for Nt in Nts:
            labels.append('%ix%i_S%i' %(Nx,Ny,Nt))
            
    f = open('params', 'a')
    writer = csv.writer(f)
    for ridx, row in enumerate(params):
        row.insert(0, labels[ridx])
        writer.writerow(row)
    f.close()


SIDX; NIDX; elem.y at SIDX; elem.y at NIDX:  0 12 0.09999999999999998 2.5


TypeError: stratification_function() missing 1 required positional argument: 'dy'

## Plot annotated heatmap in Appendix C
Note: **Restart kernel** before running these cells, otherwise the size of the figure will be affected by the settings in the above cells.

### Helper functions

In [ ]:
# Results copied from the output of the above cell.
r1 = [-3.268587369000301, -3.4825619191349686, -3.172512645518122, -3.3063692522837176, -3.199578503470238, -3.2158594027628475, -3.0687822587794673, -2.582052680613442]
r2 = [-2.805956457552443, -2.953639888637499, -3.14213442132599, -3.0557811331761258, -2.9696309329103157, -2.736561534884991, -2.5532790319588234, -3.044171754407027]
r3 = [-2.955322120900699, -2.9072880778451227, -2.926853860634512, -3.356723401291692, -3.3339811171041185, -3.3444880803125994, -3.3444880803125994, -3.3444880803125994]
r4 = [-2.4758224600266634, -2.7766153466764787, -3.2608002179719966, -3.2608002179719966, -3.2608002179719966, -3.2608002179719966, -3.2608002179719966, -3.2608002179719966]

RR = np.array([r1,r2,r3,r4])
data = np.copy(10**RR) * 10**4
data = np.around(data,2)

# skip results that are restricted by CFL_{adv}
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        skips = ((2,6),(2,7),(3,3),(3,4),(3,5),(3,6),(3,7))
        for (i,j) in skips:
            data[i,j] = np.nan

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib

# Annotated heatmap code modified from:
# https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="8%", pad=0.05)
    cax1 = cax.twinx()
    cax.yaxis.set_ticks_position('right')
    cbar = plt.colorbar(im, cax=cax, **cbar_kw)

    cbar.ax.set_ylabel(cbarlabel, rotation=0)
    cbar.ax.yaxis.set_label_coords(0.9,1.075)
    cax1.grid(False, which="major")
    for j, lab in enumerate(['',r'$-50\%$',r'$-25\%$',r'$\pm10\%$',r'$+25\%$',r'$+50\%$','']):
        cax1.text(.5, (j+0.5) / 7.0, lab, ha='center', va='center', rotation=-0)
    cax1.set_yticks([])

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-0, ha="center",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.set_xlabel("Temporal step size~[s]")
    ax.xaxis.set_label_position('top')
    ax.set_ylabel("Spatial resolution\n[number of grid points]")
    ax.grid(which="minor", color="w", linestyle='-', linewidth=4)
    ax.grid(False, which="major")
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
#     if threshold is not None:
#         threshold = im.norm(threshold)
#     else:
#         threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = mtl.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
#             kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
    
            skips = ((2,6),(2,7),(3,3),(3,4),(3,5),(3,6),(3,7))
            if (i,j) in skips:
                text = im.axes.text(j, i, "", **kw)
                texts.append('')
            else:
                if threshold is not None:
                    kw.update(color=textcolors[int(data[i, j] < threshold[0] or data[i, j] > threshold[1])])
                    
                text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
                texts.append(text)
    return texts


### Plot the annotated heatmap

In [ ]:
resos = [r'$151 \times 60$',r'$301 \times 120$',r'$601 \times 240$',r'$1201 \times 480$']
treses = [r'200',r'400',r'600',r'800',r'1000',r'1200',r'1400',r'1600']

from matplotlib import colors
divnorm=colors.TwoSlopeNorm(vmin=-3.6, vcenter=7.85, vmax=34)
norm = matplotlib.colors.BoundaryNorm([3.20, 3.93, 5.89, 7.07, 8.65, 9.82, 11.8, 34.0], 7)

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
top = cm.get_cmap('Oranges_r', 4)
middle = cm.get_cmap('Oranges_r', 1)
bottom = cm.get_cmap('Oranges', 4)

newcolors = np.vstack((top(np.linspace(0, 1, 4)[:-1]),
                       middle(np.linspace(0, 1, 1)),
                       bottom(np.linspace(0, 1, 4)[1:])
                      ))
newcmp = ListedColormap(newcolors, name='OrangeDivergence')

print(data)
plt.figure(figsize=(8,8))

ax = plt.gca()
cmap = plt.get_cmap("PRGn", 7)
cmap = newcmp
cmap.set_bad(color='white', alpha = 1.)
im, cbar = heatmap(data, resos, treses, ax=ax, cmap=cmap, cbarlabel=r"$\times 10^{-4}$", norm=norm)
texts = annotate_heatmap(im, valfmt="{x:.2f}", threshold=[3.93,11.80])
plt.tight_layout()
fld_pth = './output_plot/pub'
if not os.path.exists(fld_pth):
    os.makedirs(fld_pth)
plt.savefig("%s/heatMap.pdf" %fld_pth)
plt.show()

### Compute boundaries for each colour band of the heatmap.

In [ ]:
xp = -3.104615552507752

print(10**xp)

a = 0.5*10**xp
b = 1.5*10**xp
print(a,b)

a = 0.75*10**xp
b = 1.25*10**xp
print(a,b)

a = 0.90*10**xp
b = 1.10*10**xp
print(a,b)